# Hubbard Model

Gates for Qubitizing the Hubbard Model.

This follows section V. of the [Linear T Paper](https://arxiv.org/abs/1805.03662).

The Hubbard model is a special case of the electronic structure Hamiltonian
restricted to spins on a planar grid.

$$
H = -t \sum_{\langle p,q \rangle, \sigma} a_{p,\sigma}^\dagger a_{q,\sigma}
    + \frac{u}{2} \sum_{p,\alpha\ne\beta} n_{p, \alpha} n_{p, \beta}
$$

Under the Jordan-Wigner transformation this is

$$
\def\Zvec{\overrightarrow{Z}}
\def\hop#1{#1_{p,\sigma} \Zvec #1_{q,\sigma}}
H = -\frac{t}{2} \sum_{\langle p,q \rangle, \sigma} (\hop{X} + \hop{Y})
  + \frac{u}{8} \sum_{p,\alpha\ne\beta} Z_{p,\alpha}Z_{p,\beta}
  - \frac{u}{4} \sum_{p,\sigma} Z_{p,\sigma} + \frac{uN}{4}\mathbb{1}
$$


This model consists of a PREPARE and SELECT operation where our selection operation has indices
for p, alpha, q, and beta as well as two indicator bits U and V. There are four cases considered
in both the PREPARE and SELECT operations corresponding to the terms in the Hamiltonian:

 - $U=1$, single-body Z
 - $V=1$, spin-spin ZZ term
 - $p<q$, XZX term
 - $p>q$, YZY term.

See the documentation for `PrepareHubbard` and `SelectHubbard` for details.

In [ ]:
import cirq
import numpy as np
import cirq_qubitization as cq
import cirq_qubitization.cirq_infra.testing as cq_testing
from cirq_qubitization.jupyter_tools import display_gate_and_compilation
from typing import *

## `SelectHubbard`
The SELECT operation optimized for the 2D Hubbard model.

In contrast to the arbitrary chemistry Hamiltonian, we:
 - explicitly consider the two dimensions of indices to permit optimization of the circuits.
 - dispense with the `theta` parameter.

If neither `U` nor `V` is set we apply the kinetic terms of the Hamiltonian:

$$
-\hop{X} \quad p < q \\
-\hop{Y} \quad p > q
$$

If U is set we know $(p,\alpha)=(q,\beta)$ and apply the single-body term: $-Z_{p,\alpha}$.
If V is set we know $p=q, \alpha=0$, and $\beta=1$ and apply the spin term:
$Z_{p,\alpha}Z_{p,\beta}$


#### Parameters
 - `x_dim`: the number of sites in the x axis.
 - `y_dim`: the number of site along the y axis. 

#### Registers
 - `control[1]`: A control bit for the entire gate.
 - `U[1]`: Whether we're applying the single-site part of the potential.
 - `V[1]`: Whether we're applying the pairwise part of the potential.
 - `p_x`: First set of site indices, x component.
 - `p_y`: First set of site indices, y component.
 - `alpha[1]`: First set of sites' spin indicator.
 - `q_x`: Second set of site indices, x component.
 - `q_y`: Second set of site indices, y component.
 - `beta[1]`: Second set of sites' spin indicator.
 - `target`: The system register to apply the select operation.
 - `ancilla`: Work space. 

#### References
Section V. and Fig. 19 of https://arxiv.org/abs/1805.03662.


In [ ]:
from cirq_qubitization.cirq_algos.hubbard_model import *

x_dim, y_dim, t = 2, 2, 5
mu = 4 * t

select = cq_testing.GateHelper(
    SelectHubbard(x_dim=x_dim, y_dim=y_dim)
)
display_gate_and_compilation(select)

In [ ]:
print(cq.t_complexity(SelectHubbard(x_dim=x_dim, y_dim=y_dim)))

## `PrepareHubbard`
The PREPARE operation optimized for the 2D Hubbard model.

In contrast to the arbitrary chemistry Hamiltonian, we:
 - explicitly consider the two dimensions of indices to permit optimization of the circuits.
 - dispense with the `theta` parameter.

#### Parameters
- `x_dim`: the number of sites along the x axis.
- `y_dim`: the number of sites along the y axis.
- `t`: coefficient for XZX and YZY terms in the Hubbard model hamiltonian.
- `mu`: coefficeint for single body Z term and two-body ZZ terms in the Hubbard model hamiltonian.

#### Registers
 - `control[1]`: A control bit for the entire gate.
 - `U[1]`: Whether we're applying the single-site part of the potential.
 - `V[1]`: Whether we're applying the pairwise part of the potential.
 - `p_x`: First set of site indices, x component.
 - `p_y`: First set of site indices, y component.
 - `alpha[1]`: First set of sites' spin indicator.
 - `q_x`: Second set of site indices, x component.
 - `q_y`: Second set of site indices, y component.
 - `beta[1]`: Second set of sites' spin indicator.
 - `target`: The system register to apply the select operation.
 - `ancilla`: Work space. 

#### References
Section V. and Fig. 20 of https://arxiv.org/abs/1805.03662.

In [ ]:
prepare = cq_testing.GateHelper(
    PrepareHubbard(x_dim=x_dim, y_dim=y_dim, t=t, mu=mu)
)
display_gate_and_compilation(prepare)

In [ ]:
print(cq.t_complexity(PrepareHubbard(x_dim=x_dim, y_dim=y_dim, t=t, mu=mu)))